In [1]:
import pickle, tqdm, time, os, tensorflow_hub as hub, numpy as np, pandas as pd

**Read Data Stored Locally**

In [2]:
# All have order preserved across them
titles = pd.read_pickle('data/titles.pkl')
ids = pd.read_pickle('data/ids.pkl')
texts = pd.read_pickle('data/texts.pkl')

In [3]:
# create df
df = pd.DataFrame()
df['ids'] = ids
df['titles'] = titles
df['texts'] = texts

In [4]:
df.isna().sum()

ids           1
titles        2
texts     14189
dtype: int64

In [5]:
# drop all entries which have a null value in any of id, titles, texts
df = df.dropna().copy()
df = df.reset_index(drop=True).copy()

In [6]:
df.head()

,ids,titles,texts
0,j1f7am,AITA for not talking to my mom and not letting...,I’m a 25 yr old female and my husband is 25. W...
1,j1f71q,AITA for telling a women I am not interested a...,This happened last night and I am still proces...
2,j1f6tt,AITA for giving up on a friend who has been di...,I (19 M) have one of my busiest semester this ...
3,j1f6r4,Will I be the asshole if I drop a metaphorical...,[removed]
4,j1f4so,AITA if I only want to have sex with my girlfr...,[removed]


In [7]:
df['texts'].value_counts()

[removed]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [8]:
df['titles'].value_counts()

AITA                                                                                  871
AITA?                                                                                 283
Am I the asshole?                                                                     100
WIBTA                                                                                  88
AIs from AI Dungeon 2 to sexy to funny and one based wholly on Reddit!                 79
                                                                                     ... 
AITA for not caring about my sister crying?                                             1
AITA for kicking out my younger brother from his room                                   1
AITA My friend was sleeping around during lockdown so I called her a selfish c*nt.      1
AITA for demanding a small family run business to refund my 4 months old purchase?      1
AItA: I like air conditioning and my coworkers like working half-naked.                 1
Name: titl

In [9]:
#remove entries that have deleted or removed text
df = df[(df['texts'] != '[removed]') & (df['texts'] != '[deleted]')].copy()

In [10]:
df.shape

(490330, 3)

In [11]:
# df.to_pickle('data/clean_data.pkl')

In [12]:
NUM_DOCS = len(df)

In [13]:
# load & cache tensorflow model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embed(['testing'])
print('model cached')

model cached


In [14]:
from tensorflow.python.client import device_lib

In [15]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [16]:
get_available_gpus()

['/device:GPU:0']

**Converting Each Post From Text To Embedding**
- Start off with using the title of the post only

In [17]:
# calculates end index for a particular iteration for looping through documents in batches
def calcEndIdx(start_idx, batch_size, ndocs):
    end_idx = start_idx + batch_size
    end_idx = ndocs if end_idx > ndocs - 1 else end_idx
    return end_idx

In [22]:
# convert text to embeddings in batches (model can handle multiple texts at once)
# batch size depends on compute power
text_data = df['texts'].values # text data - can be texts col or title col
embeddings = [] # empty array to store embeddings as we iterate through docs
BATCH_SIZE = 16

for start_idx in tqdm.tqdm_notebook(range(232800, NUM_DOCS, BATCH_SIZE)):
    end_idx = calcEndIdx(start_idx, BATCH_SIZE, NUM_DOCS)
    curr_embeddings = embed(text_data[start_idx:end_idx]).numpy()
    embeddings.append(curr_embeddings)
    
#embeddings = np.concatenate(embeddings) # convert batched arrays to shape (N, Vector Size)

<ipython-input-22-0c7a183029b6>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for start_idx in tqdm.tqdm_notebook(range(232800, NUM_DOCS, BATCH_SIZE)):


  0%|          | 0/16096 [00:00<?, ?it/s]

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[16,8,5530,5530] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node StatefulPartitionedCall/StatefulPartitionedCall/EncoderTransformer/Transformer/SparseTransformerEncode/Layer_0/SelfAttention/SparseMultiheadAttention/DotProductAttention/MatMul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[StatefulPartitionedCall/StatefulPartitionedCall/EncoderTransformer/Transformer/layer_prepostprocess/layer_norm/add_1/_240]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[16,8,5530,5530] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator G

In [ ]:
len(embeddings)

In [ ]:
np.savez('data/texts_embeddings_until_232800_end_preconcat.npz', a=embeddings)